In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [2]:
from src.earthquake import utils
import config

# load training set
data = utils.read_csv(config.path_to_train)
# create list of features
features = [column for column in data.columns if column not in ['target', 'seg_id']]

In [3]:
# display importance
best_features = utils.feature_importance(data[features], data['target'], n_best=15, n_jobs=8)

  Imp | Feature
 0.15 | mfcc_15_avg
 0.12 | mfcc_5_avg
 0.07 | mfcc_4_avg
 0.05 | percentile_roll_std_10_window_100
 0.04 | percentile_roll_std_5_window_50
 0.03 | percentile_roll_std_25_window_500
 0.03 | percentile_roll_std_40_window_500
 0.02 | percentile_roll_std_20_window_1000
 0.02 | percentile_roll_std_20_window_500
 0.02 | percentile_roll_std_10_window_50
 0.02 | percentile_roll_std_25_window_100
 0.02 | percentile_roll_std_25_window_1000
 0.01 | percentile_roll_std_20_window_10
 0.01 | percentile_roll_std_10_window_500
 0.01 | percentile_roll_std_40_window_1000


In [4]:
import numpy as np
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor

# set output float precision 
np.set_printoptions(precision=3)
# init model
model = CatBoostRegressor(random_seed=0, verbose=False)
# calculate mae on folds
mae = cross_val_score(model, data[best_features], data['target'], 
    cv=5, scoring='neg_mean_absolute_error', n_jobs=8)
# print the results
print('folds: {}'.format(abs(mae)))
print('total: {:.3f}'.format(np.mean(abs(mae))))

folds: [1.982 2.333 2.379 1.266 2.362]
total: 2.064


In [5]:
ga_features = [
    'ffti_av_change_rate_roll_mean_1000', 'percentile_roll_std_30_window_50', 'skew',
    'percentile_roll_std_10_window_100', 'percentile_roll_std_30_window_50',
    'percentile_roll_std_20_window_1000', 'ffti_exp_Moving_average_30000_mean',
    'range_3000_4000', 'max_last_10000', 'mfcc_4_avg',
    'fftr_percentile_roll_std_80_window_10000', 'percentile_roll_std_1_window_100',
    'ffti_abs_trend', 'av_change_abs_roll_mean_50', 'mfcc_15_avg'
]

mae = cross_val_score(model, data[ga_features], data['target'], 
    cv=5, scoring='neg_mean_absolute_error', n_jobs=8)

print('folds: {}'.format(abs(mae)))
print('total: {:.3f}'.format(np.mean(abs(mae))))

folds: [1.973 2.313 2.357 1.262 2.334]
total: 2.048
